In [1]:
import plaidml.keras
plaidml.keras.install_backend()

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import adam
from sklearn.model_selection import train_test_split

from code.util import *

# MLP w/ Engineered Features

In [2]:
batch_size = 128
num_classes = 2
epochs = 50

In [3]:
x = load("data/largefiles/x_feat.pkl")
y = load("data/largefiles/y_onehot.pkl")

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42)

In [5]:
print(x.shape, y.shape)

(127284, 50) (127284, 2)


In [6]:
model = Sequential()
model.add(Dense(50, activation='tanh', input_shape=(50,)))
model.add(Dropout(0.2))
model.add(Dense(50, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0819 19:44:56.543683 4562654656 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 19:44:56.575459 4562654656 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 19:44:56.586928 4562654656 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 19:44:56.607791 4562654656 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:13

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 5,202
Trainable params: 5,202
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

Train on 89098 samples, validate on 38186 samples
Epoch 1/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1297 - acc: 0.9550 - val_loss: 0.1297 - val_acc: 0.9547
Epoch 2/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1298 - acc: 0.9553 - val_loss: 0.1295 - val_acc: 0.9547
Epoch 3/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1297 - acc: 0.9557 - val_loss: 0.1331 - val_acc: 0.9543
Epoch 4/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1303 - acc: 0.9552 - val_loss: 0.1284 - val_acc: 0.9551
Epoch 5/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1302 - acc: 0.9552 - val_loss: 0.1286 - val_acc: 0.9549
Epoch 6/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1303 - acc: 0.9550 - val_loss: 0.1282 - val_acc: 0.9546
Epoch 7/50
89098/89098 [==============================] - 1s 8us/step - loss: 0.1299 - acc: 0.9553 - val_loss: 0.1290 - val_acc: 0.954

In [10]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.12564320442293248
Test accuracy: 0.9567118839365213
